<a href="https://colab.research.google.com/github/lsaif/Fracture-and-Vuggy-Porosity/blob/main/FractureAndVuggyPorosity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lasio

In [11]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import os
import lasio

np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

source_dir = '/content/drive/MyDrive/FractureVuggyPorosity/data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
def LoadData(filename):
  l = lasio.read(f"/{source_dir}/{filename}")
  data_well = l.df()
  data_well['WELL'] = l.well.WELL.value
  return data_well

In [49]:
data_raw = pd.DataFrame()
for f in sorted(os.listdir(source_dir)):
  data_well = LoadData(f)
  data_raw = data_raw.append(data_well)
data_raw = data_raw.dropna()
data = data_raw

In [ ]:
data.head(5)

In [ ]:
phi_block = 0.03
threshold = 0.01
data['error'] = 50
data['PHI2'] = (data['PHIE']-phi_block)/(1-phi_block)
data['PHIF1'] = 1.5*data['RMF']/data['PROX']
data['PHIV1']= 0.1

while ((data['error'] >= threshold).sum()):
  data['PHIV2'] = data['PHI2'] - data['PHIF2']
  data['term1'] = 3*data['RMF']*(1-data['PHIV1'])
  data['term2'] = 2*data['PROX']*(1+2*data['PHIV1'])
  data['term3'] = 3*data['RMF']*data['PHIV1']
  data['PHIF2'] = data['term1']/(data['term2']-data['term3'])
  data['error'] = np.abs(data['PHIF2']-data['PHIF1'])
  data['PHIF1'] = data['PHIF2']
  data['PHIV1'] = data['PHIV2']

In [103]:
data.drop(['term1', 'term2','term3','PHIF1','PHIV1'], axis=1, inplace=True)
data.head(20)
data.to_csv('result.csv')